In [1]:
!pwd
import sys
%reload_ext autoreload
%autoreload 2
%matplotlib inline
sys.executable

C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\MLP_for_ensemble\20201124\run_objective_5l


'c:\\users\\81908\\appdata\\local\\pypoetry\\cache\\virtualenvs\\tfgpu-ehdmne1y-py3.8\\scripts\\python.exe'

In [2]:
# tensorflow2.0 + kerasでGPUメモリの使用量を抑える方法(最小限だけ使うように設定)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [4]:
import sys
sys.path.append(r'C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\MLP_for_ensemble\20201124')
import moa_MLPs_funcs

Tensorflow version 2.3.1
n_classes: 206


In [5]:
import optuna


def objective(trial):
    model_type = "5l"
    params = {}
    params["activation"] = trial.suggest_categorical("activation", ["relu", "elu"])
    params["denses"] = trial.suggest_categorical(
        "denses",
        [
            [512, 448, 384, 320, 256],
            [1024, 896, 768, 640, 512],
            [2048, 1792, 1536, 1280, 1024],
            [2560, 2048, 1524, 1012, 780],
            [512, 512, 512, 512, 512],
            [1024, 1024, 1024, 1024, 1024],
            [1024, 768, 1280, 1536, 1792],
            [1024, 1280, 1536, 768, 1792],
            [512, 768, 1280, 1536, 1792],
            [512, 1536, 1280, 768, 512],
        ],
    )
    drop_rates = []
    for i in range(6):
        drop_rate = trial.suggest_uniform(f'drop_rate{i}', 0.2, 0.8)
        drop_rates.append(drop_rate)
    params["drop_rates"] = drop_rates
    print("-" * 100)
    print(f"params: {params}")

    oof_score, Y_pred = moa_MLPs_funcs.train_and_evaluate(model_type="5l", params=params)

    return oof_score

In [6]:
%%time

n_trials = 100
#n_trials = 50

study = optuna.create_study(
    study_name="study",
    storage=f"sqlite:///study.db",
    load_if_exists=True,
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=1),
)
study.optimize(objective, n_trials=n_trials)
study.trials_dataframe().to_csv(f"objective_history.csv", index=False)
with open(f"objective_best_params.txt", mode="w") as f:
    f.write(str(study.best_params))
print(f"\nstudy.best_params:\n{study.best_params}")

[I 2020-11-24 15:01:13,689] A new study created in memory with name: study
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\optuna\distributions.py:404: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [512, 448, 384, 320, 256] which is of type list.
  warnings.warn(message)
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\optuna\distributions.py:404: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1024, 896, 768, 640, 512] which is of type list.
  warnings.warn(message)
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\optuna\distributions.py:404: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent stora

----------------------------------------------------------------------------------------------------
params: {'activation': 'elu', 'denses': [2560, 2048, 1524, 1012, 780], 'drop_rates': [0.6321946960652949, 0.20006862489040694, 0.3813995435791039, 0.2880535344902678, 0.2554031568612787, 0.3117561268266026]}



c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Epoch 1/50
19/19 - 2s - loss: 0.3572 - binary_crossentropy: 0.3556 - val_loss: 0.5598 - val_binary_crossentropy: 0.5195
Epoch 2/50
19/19 - 1s - loss: 0.0294 - binary_crossentropy: 0.0231 - val_loss: 0.1568 - val_binary_crossentropy: 0.1399
Epoch 3/50
19/19 - 1s - loss: 0.0291 - binary_crossentropy: 0.0219 - val_loss: 0.0598 - val_binary_crossentropy: 0.0492
Epoch 4/50
19/19 - 1s - loss: 0.0273 - binary_crossentropy: 0.0207 - val_loss: 0.0379 - val_binary_crossentropy: 0.0298
Epoch 5/50
19/19 - 1s - loss: 0.0266 - binary_crossentropy: 0.0205 - val_loss: 0.0300 - val_binary_crossentropy: 0.0231
Epoch 6/50
19/19 - 1s - loss: 0.0264 - binary_crossentropy: 0.0205 - val_loss: 0.0281 - val_binary_crossentropy: 0.0217
Epoch 7/50
19/19 - 1s - loss: 0.0262 - binary_crossentropy: 0.0204 - val_loss: 0.0272 - val_binary_crossentropy: 0.0210
Epoch 8/50
19/19 - 1s - loss: 0.0263 - binary_crossentropy: 0.0205 - val_loss: 0.0266 - val_binary_crossentropy: 0.0206
Epoch 9/50
19/19 - 1s - loss: 0.0262 - b

Epoch 19/50
19/19 - 1s - loss: 0.0257 - binary_crossentropy: 0.0199 - val_loss: 0.0247 - val_binary_crossentropy: 0.0189
Epoch 20/50
19/19 - 1s - loss: 0.0256 - binary_crossentropy: 0.0198 - val_loss: 0.0247 - val_binary_crossentropy: 0.0188
Epoch 21/50
19/19 - 1s - loss: 0.0256 - binary_crossentropy: 0.0197 - val_loss: 0.0246 - val_binary_crossentropy: 0.0188
Epoch 22/50
19/19 - 1s - loss: 0.0256 - binary_crossentropy: 0.0198 - val_loss: 0.0246 - val_binary_crossentropy: 0.0187
Epoch 23/50
19/19 - 1s - loss: 0.0257 - binary_crossentropy: 0.0198 - val_loss: 0.0245 - val_binary_crossentropy: 0.0187
Epoch 24/50
19/19 - 1s - loss: 0.0256 - binary_crossentropy: 0.0198 - val_loss: 0.0245 - val_binary_crossentropy: 0.0187
Epoch 25/50
19/19 - 1s - loss: 0.0256 - binary_crossentropy: 0.0198 - val_loss: 0.0245 - val_binary_crossentropy: 0.0186
Epoch 26/50
19/19 - 1s - loss: 0.0256 - binary_crossentropy: 0.0197 - val_loss: 0.0244 - val_binary_crossentropy: 0.0186
Epoch 27/50
19/19 - 1s - loss: 0

KeyboardInterrupt: 